# Using QCArchive with the OpenFF Toolkit

Here we show how to create OpenFF molecules safely from data in the QCArchive using the CMILES entries. This transformation relies on the `"canonical_isomeric_explicit_hydrogen_mapped_smiles"`.

First load up the client you wish to connect to, in this case, we use the public instance.

In [1]:
import qcportal

from openff.toolkit import Molecule

client = qcportal.PortalClient("https://api.qcarchive.molssi.org:443")

print(client.list_datasets_table())

  id  type                record_count  name
----  ----------------  --------------  ---------------------------------------------------------------------------------------
  35  torsiondrive                 454  OpenFF Fragmenter Phenyl Benchmark
  36  torsiondrive                 820  OpenFF Group1 Torsions
  41  optimization                 937  OpenFF Optimization Set 1
  42  torsiondrive                  86  Fragment Stability Benchmark
  43  optimization                1132  SMIRNOFF Coverage Set 1
  45  optimization               25500  OpenFF VEHICLe Set 1
  48  torsiondrive                 585  SMIRNOFF Coverage Torsion Set 1
  49  optimization                 189  OpenFF NCI250K Boron 1
  50  optimization               19714  OpenFF Discrepancy Benchmark 1
  57  torsiondrive                 795  OpenFF Substituted Phenyl Set 1
  68  optimization                 352  Pfizer Discrepancy Optimization Dataset 1
  69  optimization                6670  FDA Optimization Dataset 1
  

Data in the QCArchive is organized into [datasets](https://molssi.github.io/QCFractal/user_guide/datasets.html#using-datasets). Let's grab a molecule from an optimization dataset

In [2]:
dataset = client.get_dataset(
    dataset_type="optimization",
    dataset_name="Kinase Inhibitors: WBO Distributions",
)

Take an arbitrary entry from the collection.

In [3]:
entry = dataset.get_entry(entry_name=dataset.entry_names[-1])

We can view the entry in detail by looking at the dictionary representation.

In [4]:
entry.dict()

{'name': 'cs(=o)(=o)ccncc1ccc(o1)c2ccc3c(c2)c(ncn3)nc4ccc(c(c4)cl)occ5cccc(c5)f-99',
 'initial_molecule': {'schema_name': 'qcschema_molecule',
  'schema_version': 2,
  'validated': True,
  'symbols': array(['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C',
         'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C',
         'C', 'C', 'C', 'N', 'N', 'N', 'N', 'O', 'O', 'O', 'O', 'F', 'S',
         'Cl', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H',
         'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H',
         'H'], dtype='<U2'),
  'geometry': array([[ 2.89795594e+01, -5.61363284e+00, -1.25287815e+00],
         [ 1.93871961e+00, -2.32111000e-01,  6.33108960e-01],
         [ 2.68259179e+01, -4.17201102e+00, -7.72020570e-01],
         [ 3.00076085e+00,  1.52774996e+00,  2.29260082e+00],
         [ 1.49570525e+01, -1.11862575e+00,  1.34408755e+00],
         [ 1.75606324e+01, -1.34772255e+00,  1.00269997e+00],
         [ 2.88520

Now we can make a molecule using a few different input options.

In [5]:
# first make a molecule using this record object
molecule_from_entry = Molecule.from_qcschema(entry)

# we could have also used the dictionary representation of the object
molecule_from_dict = Molecule.from_qcschema(entry.dict())

assert molecule_from_entry == molecule_from_dict

molecule = molecule_from_entry

In [6]:
# first let's get the initial molecule from the database
initial_molecule = client.get_molecules(entry.initial_molecule.id)

# note that this molecule uses an object model from QCArchive, _not_ the toolkit
print(type(initial_molecule))

# we check that the molecule has been ordered to match the ordering used in the data base
# by printing out the atomic numbers of both objects in order

for atoms in zip(molecule.atoms, initial_molecule.atomic_numbers):
    print(atoms[0].atomic_number, atoms[1])
    assert atoms[0].atomic_number == atoms[1]

# can compare other things, too
print(molecule.to_hill_formula(), initial_molecule.get_molecular_formula())

# QCArchive molecules don't store all information the
# toolkit needs, like bond orders and formal charges;
# that's why there is a Molecule.from_qcschema() method at all

<class 'qcelemental.models.molecule.Molecule'>
6 6
6 6
6 6
6 6
6 6
6 6
6 6
6 6
6 6
6 6
6 6
6 6
6 6
6 6
6 6
6 6
6 6
6 6
6 6
6 6
6 6
6 6
6 6
6 6
6 6
6 6
6 6
6 6
6 6
7 7
7 7
7 7
7 7
8 8
8 8
8 8
8 8
9 9
16 16
17 17
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
C29H26ClFN4O4S C29ClFH26N4O4S


In [7]:
# we can also compare the graph representations of the molecules to make sure they are in the same order
import networkx as nx

# make a graph of the initial molecule using newtorkx and the data in the record
initial_network = nx.Graph()
for index, atomic_number in enumerate(initial_molecule.atomic_numbers):
    initial_network.add_node(index, atomic_number=atomic_number)

for bond in initial_molecule.connectivity:
    initial_network.add_edge(*bond[:2])
# now we can use the new isomorphic check to get the atom mapping
isomorphic, atom_map = Molecule.are_isomorphic(
    molecule,
    initial_network,
    return_atom_map=True,
    aromatic_matching=False,
    formal_charge_matching=False,
    bond_order_matching=False,
    bond_stereochemistry_matching=False,
    atom_stereochemistry_matching=False,
)

# we can check if the graph was found to be isomorphic and whether or not the
# atom mappings are in the same order
assert isomorphic
print(atom_map)
for index1, index2 in atom_map.items():
    assert index1 == index2

{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12, 13: 13, 14: 14, 15: 15, 16: 16, 17: 17, 18: 18, 19: 19, 20: 20, 21: 21, 22: 22, 23: 23, 24: 24, 25: 25, 26: 26, 27: 27, 28: 28, 29: 29, 30: 30, 31: 31, 32: 32, 33: 33, 34: 34, 35: 35, 36: 36, 37: 37, 38: 38, 39: 39, 40: 40, 41: 41, 42: 42, 43: 43, 44: 44, 45: 45, 46: 46, 47: 47, 48: 48, 49: 49, 50: 50, 51: 51, 52: 52, 53: 53, 54: 54, 55: 55, 56: 56, 57: 57, 58: 58, 59: 59, 60: 60, 61: 61, 62: 62, 63: 63, 64: 64, 65: 65}


In [8]:
mol = Molecule.from_qcschema(entry)
mol

NGLWidget()

In [9]:
# OpenFF Toolkit `Molecule` objects can be converted back into QCArchive molecules,
# as long as there are conformer(s)

qc_molecule = molecule.to_qcschema()

qc_molecule

NGLWidget()

This transformation unlocks functionality of [QCEngine](https://molssi.github.io/QCEngine/) (computing energies, gradients, hessians, etc. with a variety of different methods).

Here we will try and compute the energy using RDKit (only run this cell if QCEngine is installed.) 

In [10]:
import qcengine

# set up the RDKit task
rdkit_task = {
    "schema_name": "qcschema_input",
    "schema_version": 2,
    "molecule": qc_molecule,
    "driver": "energy",
    "model": {"method": "uff", "basis": None},
    "keywords": {"scf_type": "df"},
}

# now lets compute the energy using qcengine and RDKit and print the result
result = qcengine.compute(rdkit_task, "rdkit")

In [11]:
# note the result is in QC units of hartrees
print(result.return_result)

0.05930479138457709
